In [ ]:
pip install ipykernel

kinguin webscraper, v1
- shows prices only
- limitation: ambiguous platform (doesn't differentiate between platforms)
- limitation: ambiguous product type (doesn't differentiate between dlc and game key)
- limitation: ambiguous region lock (doesn't differentiate between different region locks)
- limitation: doesn't show OOS products

In [1]:
#v1 kinguin scraper

##############
# import modules
##############
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re

############
# get text and game name
# $future task: enable chrome headless mode, per https://stackoverflow.com/questions/46920243/how-to-configure-chromedriver-to-initiate-chrome-browser-in-headless-mode-throug/49582462#49582462
# $future task: allow user to specify which game to search
############
print('What game are you searching for? ')
# game_title = input().lower()
game_title = 'Baldur\'s Gate 3'.lower()
print(game_title)
print()
url_game_title = game_title.replace(' ', '%20').replace('\'', '%27').title()
url = 'https://www.kinguin.net/listing?phrase=' + url_game_title

driver=webdriver.Chrome() #bypass non-valid Chrome header by launching Chrome, per https://stackoverflow.com/questions/53918187/python-requests-to-continue-your-browser-has-to-accept-cookies-and-has-to-hav
driver.get(url)
soup = BeautifulSoup(driver.page_source,'lxml')
text = soup.get_text().lower()

########################################################
# search for game title
# $future task: filter out OOS items
########################################################
# starting_phrase = 'Baldur\'s'
# ending_char = '$'

# # Create a regular expression pattern to match the desired substring
# pattern = re.escape(starting_phrase) + r'.*?[' + re.escape(ending_char) + r']'

# # Use re.findall to find all matches in the string
# matches = re.findall(pattern, text)

# # Print the matched substrings
# for match in matches:
#     print(match)

#######################
# search for $ amount
#######################
char_to_find = '$'
indices = [i for i in range(len(text)) if text[i] == char_to_find]

raw_prices = [text[index+1:index+7] for index in indices]
cleaned_prices = [re.sub('[a-zA-Z]', '', item) for item in raw_prices]
float_prices = [float(item) for item in cleaned_prices if item.replace('.', '').isdigit()]
sorted_prices = sorted(float_prices)

print('Kinguin\'s <<' + game_title + '>> prices')
print(sorted_prices)

######################
# testing.py
# create unit tests
######################

What game are you searching for? 
baldur's gate 3

Kinguin's <<baldur's gate 3>> prices
[9.29, 12.93, 21.85, 27.69, 38.27, 48.7, 71.98, 71.98, 71.98, 71.99]


kinguin webscraper, v2
- feature: shows product, platform, product type, price, site
- feature: shows OOS products

In [1]:
#v2 kinguin scraper

##############
# import modules
##############
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import pandas as pd

############
# get text and game name
# $future task: enable chrome headless mode, per https://stackoverflow.com/questions/46920243/how-to-configure-chromedriver-to-initiate-chrome-browser-in-headless-mode-throug/49582462#49582462
############
print('What game are you searching for? ')
# game_title = input().lower()
game_title = 'Baldur\'s Gate 3'.lower()
print(game_title)
print()
url_game_title = game_title.replace(' ', '%20').replace('\'', '%27').title()
url = 'https://www.kinguin.net/listing?phrase=' + url_game_title

driver = webdriver.Chrome() #bypass non-valid Chrome header by launching Chrome, per https://stackoverflow.com/questions/53918187/python-requests-to-continue-your-browser-has-to-accept-cookies-and-has-to-hav
driver.get(url)
soup = BeautifulSoup(driver.page_source,'lxml')
text = soup.get_text().lower()

# capture region lock, game system, product type, price
pattern = re.escape(game_title) + r"(.*?)" + re.escape('add to cart')
matches = re.findall(pattern, text)

# clean matches by price
price_clean = []
for i in matches:
    if 'out of stock' in i:
        oos_match = i.replace('out of stock', '`out of stock')
        price_clean.append(oos_match)
    if '$' in i:
        instock_match = i.replace('$', '`$')
        price_clean.append(instock_match)

# clean matches by platform
platform_clean = []
for i in price_clean:
    if 'xbox one / xbox series x|s / pc' in i:
        xbox_windows_match = i.replace('xbox one / xbox series x|s / pc', '`xbox one / xbox series x|s / pc`')
        platform_clean.append(xbox_windows_match)
    elif 'pc / xbox one / xbox series x|s' in i:
        xbox_windows_match = i.replace('pc / xbox one / xbox series x|s', '`xbox one / xbox series x|s / pc`')
        platform_clean.append(xbox_windows_match)
    elif 'xbox one / series x|s / windows 10' in i:
        xbox_series_windows_match = i.replace('xbox one / series x|s / windows 10', '`xbox one / xbox series x|s / pc`')
        platform_clean.append(xbox_series_windows_match)
    elif 'xbox one / xbox series x|s' in i:
        xbox_match = i.replace('xbox one / xbox series x|s', '`xbox one / xbox series x|s`')
        platform_clean.append(xbox_match)
    elif 'xbox one/pc' in i:
        xbox_windows_match = i.replace('xbox one/pc', '`xbox one / pc`')
        platform_clean.append(xbox_windows_match)
    elif 'xbox one / pc' in i:
        xbox_windows_match = i.replace('xbox one / pc', '`xbox one/ pc`')
        platform_clean.append(xbox_windows_match)
    elif 'xbox one / windows 10' in i:
        xbox_windows_match = i.replace('xbox one / windows 10', '`xbox one / pc`')
        platform_clean.append(xbox_windows_match)

    elif 'xbox one' in i:
        xbox_match = i.replace('xbox one', '`xbox one`')
        platform_clean.append(xbox_match)
    elif 'xbox 360' in i:
        xbox_match = i.replace('xbox 360', '`xbox 360`')
        platform_clean.append(xbox_match)
    
    elif 'steam' in i:
        steam_match = i.replace('steam', '`steam`')
        platform_clean.append(steam_match)
        
    elif 'gog' in i:
        gog_match = i.replace('gog', '`gog`')
        platform_clean.append(gog_match)

    elif 'ps4' in i:
        playstation_match = i.replace('ps4', '`playstation 4`')
        platform_clean.append(playstation_match)
    elif 'ps5' in i:
        playstation_match = i.replace('ps5', '`playstation 5`')
        platform_clean.append(playstation_match)
    elif 'playstation 5' in i:
        playstation_match = i.replace('playstation 5', '`playstation 5`')
        platform_clean.append(playstation_match)
    
    elif 'nintendo 3ds' in i:
        nintendo_match = i.replace('nintendo 3ds', '`nintendo 3ds`')
        platform_clean.append(nintendo_match)
    elif 'nintendo switch' in i:
        nintendo_match = i.replace('nintendo switch', '`nintendo switch`')
        platform_clean.append(nintendo_match)

    else:
        platform_clean.append(i)

df = pd.DataFrame(platform_clean) #create dataframe
df1 = df[0].str.split('`', expand=True) #use '`' as column separator
df1.columns = ['product', 'platform', 'product type', 'price'] # add column titles
df1 = df1.map(lambda x: x.strip() if isinstance(x, str) else x) # remove all leading and trailing spaces
df1['product'] = df1['product'].str.replace('-', '') # remove - from 'region'
df1['platform'] = df1['platform'].str.replace('gog', 'gog.com')
df1['product type'] = df1['product type'].str.replace('\d+', '', regex=True) # remove digits from 'product type'
df1['product type'] = df1['product type'].str.replace(r'\.+$', '', regex=True) # remove trailing periods from 'product type'
df1['price'] = df1['price'].str.replace('$', '') # remove $ from 'prices'
df1 = df1.map(lambda x: x.strip() if isinstance(x, str) else x) # remove all leading and trailing spaces
df1['site'] = 'Kinguin'

print('Kinguin\'s <<' + game_title + '>>')
kinguin = df1.sort_values(by=['platform', 'price'])
display(kinguin)

######################
# testing.py
# create unit tests
###################### C:\Program Files (x86)\Microsoft Visual Studio\2022\BuildTools\VC\Tools\MSVC\14.37.32822\lib\x86\uwp

What game are you searching for? 
baldur's gate 3

Kinguin's <<baldur's gate 3>>


,product,platform,product type,price,site
5,digital goodie pack,gog.com,account,38.16,Kinguin
4,,gog.com,cd key,out of stock,Kinguin
14,,playstation 5,account pixelpuffin.net activation link,48.56,Kinguin
2,eu,playstation 5,cd key,98.00,Kinguin
7,,playstation 5,account,out of stock,Kinguin
9,digital deluxe edition upgrade dlc,steam,altergift,12.89,Kinguin
10,,steam,account,22.00,Kinguin
0,digital deluxe edition,steam,account,27.15,Kinguin
11,,steam,altergift,71.50,Kinguin
13,row,steam,altergift,71.50,Kinguin
